In [1]:
import math

import pandas as pd
import numpy as np
from keras import Input,backend
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, matthews_corrcoef

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv("../datasets/train_test_data.csv",
                           header=0, parse_dates=[0], index_col=0)

#params for generator
label_index = len(dataset.columns) -1


In [3]:
#hyperparamters
batch_size=64

#params for generator
delay=0
step=1 # 1 timestep = 1 day
lookback=10

#ratio for train/val/test split
train_ratio=0.7
val_ratio=0.15

train_max_idx = math.ceil(len(dataset)*train_ratio)
val_max_idx = math.ceil(len(dataset)*(train_ratio+val_ratio))

# 1 step = 1 batche of samples 
train_steps = (train_max_idx+1) // batch_size
val_steps =  (val_max_idx - train_max_idx - lookback) // batch_size
test_steps = (len(dataset) - val_max_idx - lookback) // batch_size

In [4]:
def generator(data, label_index, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=64, step=1):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index: 
                i = min_index + lookback #reset 'i'
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           (data.shape[-1] - 1)))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = np.delete(data[indices], label_index, axis=1) # an np array without the label col
            targets[j] = data[rows[j] + delay][label_index]
        yield samples, targets

In [5]:
#init generators
train_rand_gen = generator(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=train_max_idx,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)

train_gen = generator(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=train_max_idx,
                      shuffle=False,
                      step=step, 
                      batch_size=batch_size)


val_gen = generator(dataset.to_numpy(),
                    label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=train_max_idx+1,
                      max_index=val_max_idx,
                      shuffle=False,
                      step=step, 
                      batch_size=batch_size)

test_gen = generator(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=val_max_idx+1,
                      max_index=None,
                      shuffle=False,
                      step=step, 
                      batch_size=batch_size)

In [6]:
def print_metric(y_true, Y_pred):
    print('F1 score: %f' % f1_score(y_true, Y_pred))
    print('precision score: %f' % precision_score(y_true, Y_pred))
    print('recall score: %f' % recall_score(y_true, Y_pred))
    print('accuracy score: %f' % accuracy_score(y_true, Y_pred))
    print('matthews_corrcoef: %f' % matthews_corrcoef(y_true, Y_pred))
    print('\nConfusion matrix:')
    print(confusion_matrix(y_true, Y_pred, labels=[0,1]))

In [7]:
def probs_to_binary_classes(preds, threshold=0.5):
    """
    preds: np array
    threshold: scalar
    """
    return np.where(preds > threshold, 1, 0)

In [8]:
def print_unique_counts(x):
    unique, counts = np.unique(x, return_counts=True)
    print(np.asarray((unique, counts)).T)


In [9]:
#build s FC model from the book

input_shape = (lookback//step, dataset.shape[-1] - 1)

model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

# #fit the model
model.fit_generator(train_rand_gen,
                    steps_per_epoch=train_steps,
                    epochs=50, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8032      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 8,065
Trainable params: 8,065
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
61/61 [==============================] - 0s 8ms/step - loss: 0.5478 - acc: 0.7551
Epoch 2/50
61/61 [==============================] - 0s 6ms/step - loss: 0.5308 - acc: 0.7531
Epoch 3/50
61/61 [==============================] - 0s 6ms/step - loss: 0.5227 - acc: 0.7485
Epoch 4/50
61/61 [==============================] - 0s 5ms/step - loss: 0.5201 - acc: 0.7602
Epoch 5/50
61/61 [=========

In [10]:
def generator_to_samples_and_targets(generator, steps):
    count=0
    X, Y = [], []
    for samples, targets in generator:
        if count >= steps:
            break;
        else:
            count+=1

        X.append(samples)
        Y.append(targets)

    return np.concatenate(X, axis=0), np.concatenate(Y, axis=0)


In [11]:
neg = 2762+158
pos = 177+807
print("We have {0} neg cases and {1} pos cases from train data".format(neg,pos))
print("the common sense baseline (accuracy score) is {0}".format(neg/(pos+neg)))

We have 2920 neg cases and 984 pos cases from train data
the common sense baseline (accuracy score) is 0.7479508196721312


In [12]:
#Make predictions for train set
    
X, Y = generator_to_samples_and_targets(train_gen, train_steps)    
Y_pred = model.predict(X)
print_metric(Y, probs_to_binary_classes(Y_pred))

F1 score: 0.500673
precision score: 0.741036
recall score: 0.378049
accuracy score: 0.809939
matthews_corrcoef: 0.432616

Confusion matrix:
[[2790  130]
 [ 612  372]]


In [13]:
#Make predictions from dev set
X, Y = generator_to_samples_and_targets(val_gen, val_steps)    
Y_pred = model.predict(X)
print_metric(Y, probs_to_binary_classes(Y_pred))


F1 score: 0.000000
precision score: 0.000000
recall score: 0.000000
accuracy score: 0.985577
matthews_corrcoef: -0.004015

Confusion matrix:
[[820   1]
 [ 11   0]]


In [14]:
#Make predictions for test set
X, Y = generator_to_samples_and_targets(test_gen, test_steps)    
Y_pred = model.predict(X)
print_metric(Y, probs_to_binary_classes(Y_pred))


F1 score: 0.000000
precision score: 0.000000
recall score: 0.000000
accuracy score: 0.963942
matthews_corrcoef: 0.000000

Confusion matrix:
[[802   0]
 [ 30   0]]


/home/kevin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kevin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kevin/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
